Install Required Package

In [5]:
# Install Required Packages (if needed)

# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re


Authenticate

In [6]:


try:
    auth.authenticate_user()
    creds, _ = google.auth.default()
    docs_service = build('docs', 'v1', credentials=creds)
except Exception as e:
    print("ERROR: Please Login to Continie")
    raise SystemExit("Stopping script due to authentication error.")

Categorizing Markdown Content

In [7]:
#Markdown content to parse
markdown_content = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


def detect_heading_level(line):
    """
    returns heading level 1,2,3 if line starts with # character, otherwise 0
    """
    if line.startswith('# '):
        return 1
    elif line.startswith('## '):
        return 2
    elif line.startswith('### '):
        return 3
    return 0

def clean_heading(line):
    """
    Remove markdown symbol
    """
    return line.lstrip('# ').strip()

def detect_checkbox(line):
    """
    Identify if line is a checkbox item.
    Returns tuple (is_checkbox, text_without_marker, bullet_level)
    """
    bullet_level = 0
    temp = line
    while temp.startswith('  '):
        bullet_level += 1
        temp = temp[2:]

    checkbox_pattern = r'^(-|\*)\s*\[\s*\]\s+(.*)'
    match = re.match(checkbox_pattern, temp.strip())
    if match:
        return True, match.group(2), bullet_level
    return False, line, bullet_level

def detect_bullet(line):
    """
    Identify if line is bullet
    Returns tuple (is_bullet, text_without_marker, bullet_level)
    """
    bullet_level = 0
    temp = line
    while temp.startswith('  '):
        bullet_level += 1
        temp = temp[2:]

    bullet_pattern = r'^(-|\*)\s+(.*)'
    match = re.match(bullet_pattern, temp.strip())
    if match:
        return True, match.group(2), bullet_level
    return False, line, bullet_level

def process_mentions(text):
    """
    Find  mentions, return list of (substring, is_mention)
    """
    parts = []
    tokens = re.split(r'(@\w+)', text)
    for token in tokens:
        if token.startswith('@'):
            parts.append((token, True))
        else:
            parts.append((token, False))
    return parts

def get_indent_points(bullet_level):
    """
    Return indentation in points for each bullet nesting level in the multiple of 36
    """
    return 36 * bullet_level


# Create a doc and later batch update it
doc_id = None
try:
    new_doc_body = {'title': 'Automated: Product Team Sync - Sajal Regmi'}
    new_doc = docs_service.documents().create(body=new_doc_body).execute()
    doc_id = new_doc.get('documentId')
    print(f"Created doc with ID: {doc_id}")
except HttpError as e:
    print("ERROR: Failed to create new Google Doc.")
    print(f"Reason: {e}")
    raise SystemExit("Stopping script due to Doc creation error.")


DOC Creation Work Flow

In [14]:
# We will build the request payload and gradually append to it
requests = []
current_index = 1
lines = markdown_content.split('\n')
footer_section = False

for raw_line in lines:
    line = raw_line.rstrip('\r')

    if line.strip().startswith('---'): # If footer, stop
        requests.append({
            'insertText': {
                'location': {'index': current_index},
                'text': '\n'
            }
        })
        current_index += 1
        footer_section = True
        continue

    # If line is empty, just insert a newline
    if not line.strip():
        requests.append({
            'insertText': {
                'location': {'index': current_index},
                'text': '\n'
            }
        })
        current_index += 1
        continue

    heading_level = detect_heading_level(line)
    if heading_level > 0:
        heading_text = clean_heading(line)
        insert_text = heading_text + '\n'
        requests.append({
            'insertText': {
                'location': {'index': current_index},
                'text': insert_text
            }
        })
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': current_index,
                    'endIndex': current_index + len(insert_text)
                },
                'paragraphStyle': {
                    'namedStyleType': f'HEADING_{heading_level}'
                },
                'fields': 'namedStyleType'
            }
        })
        current_index += len(insert_text)
        continue

    is_checkbox, checkbox_text, bullet_level = detect_checkbox(line)
    if is_checkbox:
        insert_text = checkbox_text + '\n'
        requests.append({
            'insertText': {
                'location': {'index': current_index},
                'text': insert_text
            }
        })
        requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': current_index,
                    'endIndex': current_index + len(insert_text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })
        if bullet_level > 0:
            indent = get_indent_points(bullet_level)
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': current_index,
                        'endIndex': current_index + len(insert_text)
                    },
                    'paragraphStyle': {
                        'indentStart': {
                            'magnitude': indent,
                            'unit': 'PT'
                        },
                        'indentFirstLine': {
                            'magnitude': indent,
                            'unit': 'PT'
                        }
                    },
                    'fields': 'indentStart,indentFirstLine'
                }
            })

        mention_parts = process_mentions(checkbox_text)
        char_cursor = current_index
        for text_part, is_mention in mention_parts:
            part_len = len(text_part)
            start_pos = char_cursor
            end_pos = start_pos + part_len
            if is_mention:
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': start_pos,
                            'endIndex': end_pos
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 1.0}
                                }
                            }
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })
            char_cursor = end_pos

        current_index += len(insert_text)
        continue

    is_bullet, bullet_text, bullet_level = detect_bullet(line)
    if is_bullet:
        insert_text = bullet_text + '\n'
        requests.append({
            'insertText': {
                'location': {'index': current_index},
                'text': insert_text
            }
        })
        requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': current_index,
                    'endIndex': current_index + len(insert_text)
                },
                'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })
        if bullet_level > 0:
            indent = get_indent_points(bullet_level)
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': current_index,
                        'endIndex': current_index + len(insert_text)
                    },
                    'paragraphStyle': {
                        'indentStart': {
                            'magnitude': indent,
                            'unit': 'PT'
                        },
                        'indentFirstLine': {
                            'magnitude': indent,
                            'unit': 'PT'
                        }
                    },
                    'fields': 'indentStart,indentFirstLine'
                }
            })

        mention_parts = process_mentions(bullet_text)
        char_cursor = current_index
        for text_part, is_mention in mention_parts:
            part_len = len(text_part)
            start_pos = char_cursor
            end_pos = start_pos + part_len
            if is_mention:
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': start_pos,
                            'endIndex': end_pos
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 1.0}
                                }
                            }
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })
            char_cursor = end_pos

        current_index += len(insert_text)
        continue

    insert_text = line + '\n'
    requests.append({
        'insertText': {
            'location': {'index': current_index},
            'text': insert_text
        }
    })

    if footer_section:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': current_index,
                    'endIndex': current_index + len(insert_text)
                },
                'textStyle': {
                    'italic': True
                },
                'fields': 'italic'
            }
        })

    mention_parts = process_mentions(line)
    char_cursor = current_index
    for text_part, is_mention in mention_parts:
        part_len = len(text_part)
        start_pos = char_cursor
        end_pos = start_pos + part_len
        if is_mention:
            requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': start_pos,
                        'endIndex': end_pos
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {'red': 0, 'green': 0, 'blue': 1.0}
                            }
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })
        char_cursor = end_pos

    current_index += len(insert_text)


try:
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
    print("Markdown content successfully inserted into the new Google Doc!")
    print("Check your Google Drive for a document titled 'Automated: Product Team Sync - Sajal Regmi'.")
except HttpError as e:
    print("ERROR: Failed to batchUpdate the Google Doc with Markdown content.")
    raise

Created doc with ID: 1wHHlOp4yf3PxW9djUue3jSClKkBqWu4MwMLfl37ZHGI
Markdown content successfully inserted into the new Google Doc!
Check your Google Drive for a document titled 'Automated: Product Team Sync - Sajal Regmi'.
